# Executive Order Final Dataset

Merge some results into the final dataset.

In [7]:
import pandas
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

## Combine Datasets

Combine some datasets so we can understand which presidents and parties are in each cluster.

In [12]:
# load each original EO and do a quick word count
eo_paths = os.listdir('data/executive_orders/raw/')
eo_paths.sort()

def count_words(path):
    with open(f'data/executive_orders/raw/{path}') as f:
        return [path.split('.')[0], len(f.read().split())]

word_counts = []
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(count_words, path) for path in eo_paths]

    for future in as_completed(futures):
        word_counts.append(future.result())
        
word_counts_df = pandas.DataFrame(word_counts, columns=['name', 'word_count'])
word_counts_df

,name,word_count
0,executive-order-10043-changing-the-name-the-pr...,168
1,executive-order-10019-excusing-federal-employe...,233
2,executive-order-10009-revoking-part-executive-...,479
3,executive-order-10027-extension-trust-periods-...,241
4,executive-order-10021-further-exemption-harry-...,185
...,...,...
10434,executive-order-tendering-thanks-admiral-farra...,167
10435,executive-order-withdrawing-navajo-territories,231
10436,executive-order-withdrawing-sioux-indian-land,161
10437,order-the-president-death-zachary-taylor,125


In [24]:
clusters_df = pandas.read_csv('data/executive_orders/clusters.csv')
clusters_df['name'] = clusters_df['path'].apply(lambda x: x.split('/')[-1].split('.')[0])
clusters_df.drop(columns=['Unnamed: 0'], inplace=True)

# add work counts
combined_df = clusters_df.merge(word_counts_df, on='name')

# add sentiment which also contains the president
sentiment_df = pandas.read_csv('data/executive_orders/sentiment.csv')
combined_df = combined_df.merge(sentiment_df, on='name')

# join with president's metadata
presidents_df = pandas.read_csv('data/executive_orders/presidents.csv')
combined_df = combined_df.merge(presidents_df, on='president')

# add cluster names
cluster_analysis_df = pandas.read_json('data/executive_orders/refined_clusters.json')
cluster_analysis_df = cluster_analysis_df[['cluster', 'title']].rename(columns={'title': 'cluster_title'})
combined_df = combined_df.merge(cluster_analysis_df, on='cluster')

# join with terms by president and date field truncated to year
terms_df = pandas.read_csv('data/executive_orders/terms.csv')
terms_df['year'] = terms_df['year'].astype(str)
combined_df['year'] = combined_df['date'].apply(lambda x: x.split('-')[0])
combined_df = combined_df.merge(terms_df, on=['year', 'president'])

# drop some stuff we don't care about
combined_df.drop(columns=['path', 'duration', 'year'], inplace=True)

# set all to floats for consistency
for col in combined_df.columns:
    if col.startswith('initial') or col.startswith('revised'):
        combined_df[col] = combined_df[col].astype(float)

# write to csv
combined_df.to_csv('data/executive_orders/combined.csv', index=False)

combined_df

,cluster,tsne_1,tsne_2,name,word_count,president,president_byline,date,title,initial_joy,...,revised_immigration,revised_executive_power,revised_federalism,revised_security_liberty,revised_economic_regulation,revised_social_inclusion,presidential order,party,cluster_title,term_number
0,40,-24.312270,12.092871,executive-order-5855-assignment-frequencies-go...,943,Herbert Hoover,31st President of the United States: 1929 ‐ 1933,1932-06-06,Executive Order 5855-Assignment of Frequencies...,1.0,...,0.0,3.0,3.0,1.0,2.0,0.0,31,Republican,Executive Branch Organization: Authority and C...,36
1,43,-14.963946,45.930885,executive-order-6756-c-approval-code-fair-comp...,486,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1934-06-29,Executive Order 6756-C-Approval of Code of Fai...,1.0,...,0.0,3.0,4.0,0.0,5.0,0.0,32,Democratic,Executive Branch Advisory Groups: Formation an...,37
2,0,-43.212210,-42.001114,executive-order-taking-military-possession-rai...,141,Abraham Lincoln,16th President of the United States: 1861 ‐ 1865,1862-05-25,Executive Order-Taking Military Possession of ...,1.0,...,0.0,3.0,4.0,3.0,4.0,0.0,16,Republican,Civil War to Early 20th Century Presidential a...,19
3,0,-47.932255,-36.022755,executive-order-concerning-blockade,138,Abraham Lincoln,16th President of the United States: 1861 ‐ 1865,1862-11-12,Executive Order-Concerning Blockade,1.0,...,0.0,1.0,0.0,2.0,0.0,0.0,16,Republican,Civil War to Early 20th Century Presidential a...,19
4,11,-25.279959,-35.540390,executive-order-6801-designating-the-honorable...,202,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1934-07-27,Executive Order 6801-Designating the Honorable...,1.0,...,0.0,2.0,3.0,0.0,0.0,0.0,32,Democratic,Adjustments to Existing Executive Orders and R...,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10394,19,-3.141809,-89.441460,executive-order-7935-exemption-j-humbird-smith...,193,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1938-07-16,Executive Order 7935-Exemption of J. Humbird S...,2.0,...,0.0,1.0,0.0,0.0,0.0,0.0,32,Democratic,Presidential Exemptions to Mandatory Retirement,38
10395,24,-77.745210,42.446953,executive-order-7280-revocation-part-executive...,184,Franklin D. Roosevelt,32nd President of the United States: 1933 ‐ 1945,1936-02-01,Executive Order 7280-A-Revocation in Part of E...,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,32,Democratic,Land Withdrawal and Resurvey Orders in Western...,37
10396,29,7.557537,-15.218990,executive-order-10623-amendment-certain-provis...,787,Dwight D. Eisenhower,34th President of the United States: 1953 ‐ 1961,1955-07-23,Executive Order 10623-Amendment of Certain Pro...,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,34,Republican,Military and Veteran Personnel: Regulation and...,42
10397,14,-63.507320,0.855812,executive-order-5472-withdrawal-public-lands-p...,283,Herbert Hoover,31st President of the United States: 1929 ‐ 1933,1930-10-27,Executive Order 5472-Withdrawal of Public Land...,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,31,Republican,California and Nevada Public Land Management,36


In [3]:
combined = pandas.read_csv('data/executive_orders/combined.csv')

# group by cluster and create a column of array of unique presidents and parties
grouped = combined.groupby('cluster').agg({
    'president': lambda x: list(set(x)),
    'party': lambda x: list(set(x)),
    'cluster_title': 'first'
}).reset_index()
grouped

,cluster,president,party,cluster_title
0,0,"[Franklin D. Roosevelt, Abraham Lincoln, Milla...","[Democratic, No Party, Republican, Whig]",Civil War to Early 20th Century Presidential a...
1,1,"[George Bush, Franklin D. Roosevelt, Richard N...","[Democratic, Republican]",Executive Order Amendments: Government Structu...
2,2,"[Franklin D. Roosevelt, Warren G. Harding, Wil...","[Democratic, Republican]",Alaskan Land Reallocations for Development and...
3,3,"[Franklin D. Roosevelt, Richard Nixon, Harry S...","[Democratic, Republican]",Government Authorized Tax Return Inspections
4,4,"[Franklin D. Roosevelt, William Howard Taft, R...","[Democratic, Republican]",Government Land Management and Railway Labor R...
5,5,"[Franklin D. Roosevelt, George Bush, Richard N...","[Democratic, Republican]",Delegation of Presidential Authority to Govern...
6,6,"[Franklin D. Roosevelt, Abraham Lincoln, Richa...","[Democratic, Republican]",Executive Actions: Directing National Policy a...
7,7,"[Franklin D. Roosevelt, George Bush, Richard N...","[Democratic, Republican]",Restricted Air and Sea Zones for National Defense
8,8,"[George Bush, Barack Obama, Ulysses S. Grant, ...","[Democratic, Republican]",Presidential Directives on National and Intern...
9,9,"[Franklin D. Roosevelt, Herbert Hoover]","[Democratic, Republican]",Presidential Exemptions to Mandatory Retirement
